In [ ]:
from deepface import DeepFace


In [ ]:
import tensorflow as tf 

In [ ]:
import cv2, numpy,datetime, pygame, keyboard
import xlwings as xw

In [ ]:
model ='liveness.model'
model =tf.keras.models.load_model(model)

In [ ]:
#xlwings

workbook = xw.Book('attendance.xlsx')
sheet_name = datetime.date.today().isoformat()

try:
    worksheet=workbook.sheets(sheet_name)
except:
    worksheet = workbook.sheets.add(sheet_name)


In [ ]:
#xlwings

worksheet.range('A1').value = 'STUDENT'
worksheet.range('B1').value = 'DATE'
worksheet.range('C1').value = 'TIME'

student=[]

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pygame
from deepface import DeepFace
import datetime
# import xlwings as xw  # Uncomment if using Excel

# Initialize camera
cap = cv2.VideoCapture(0)

# Initialize pygame
pygame.init()
pg_txt = (39, 112, 50)
pg_bg = (154, 179, 157)

# Pygame setup
pygame_font = pygame.font.Font('freesansbold.ttf', 32)
screen = pygame.display.set_mode((960, 540))

# Load background image
background = pygame.image.load('images.jpeg')
background = pygame.transform.rotozoom(background, 0, 1)
screen.blit(background, (0, 0))

# Date setup
today = datetime.date.today()
t0 = today.day
students = []

# Load your trained liveness detection model
model = tf.keras.models.load_model('liveness.model')

# Uncomment and use Excel if needed
# workbook = xw.Book()
# worksheet = workbook.sheets.add(str(today))

running = True

while running:
    moment = datetime.datetime.now()
    hour = moment.hour
    minute = moment.minute
    day = moment.day
    month = moment.month
    year = moment.year

    date = f"{day}-{month}-{year}"
    time = f"{hour}:{minute}"

    if day != t0:
        t0 = day
        students = []
        # worksheet = workbook.sheets.add(date)
        # worksheet.range('A1').value = 'STUDENT'
        # worksheet.range('B1').value = 'DATE'
        # worksheet.range('C1').value = 'TIME'

    ret, frame = cap.read()
    if not ret:
        break

    try:
        res = DeepFace.find(frame, db_path='./Database', enforce_detection=False, model_name='VGG-Face')

        if len(res[0]['identity']) > 0:
            name = res[0]['identity'][0].split('/')[1].split('\\')[1]
            xmin = int(res[0]['source_x'][0])
            ymin = int(res[0]['source_y'][0])
            w = int(res[0]['source_w'][0])
            h = int(res[0]['source_h'][0])
            xmax = xmin + w
            ymax = ymin + h

            # Liveness detection
            img = frame[ymin:ymax, xmin:xmax]
            img = cv2.resize(img, (32, 32))
            img = img.astype('float') / 255.0
            img = tf.keras.preprocessing.image.img_to_array(img)
            img = np.expand_dims(img, axis=0)

            liveness = model.predict(img)
            liveness = liveness[0].argmax()

            if liveness == 1:
                cv2.putText(frame, name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)

                # Render name, date, time on screen
                n_txt = pygame_font.render(name, True, pg_txt, pg_bg)
                d_txt = pygame_font.render(date, True, pg_txt, pg_bg)
                t_txt = pygame_font.render(time, True, pg_txt, pg_bg)

                screen.blit(background, (0, 0))
                screen.blit(n_txt, n_txt.get_rect(center=(760, 270)))
                screen.blit(d_txt, d_txt.get_rect(center=(760, 310)))
                screen.blit(t_txt, t_txt.get_rect(center=(760, 350)))

                # Convert frame to RGB for pygame
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_surface = pygame.image.frombuffer(frame_rgb.tobytes(), frame_rgb.shape[1::-1], 'RGB')
                frame_surface = pygame.transform.rotozoom(frame_surface, 0, 1)

                # Optional rounded border
                size = frame_surface.get_size()
                rect_img = pygame.Surface(size, pygame.SRCALPHA)
                pygame.draw.rect(rect_img, (255, 255, 255), (0, 0, *size), border_radius=10)

                # Overlay image and border
                frame_surface = frame_surface.copy().convert_alpha()
                frame_surface.blit(rect_img, (0, 0), special_flags=pygame.BLEND_RGB_MIN)

                screen.blit(frame_surface, (20, 90))

                # Record attendance
                if name not in students:
                    students.append(name)
                    # worksheet.range(f'A{len(students)+1}').value = [name, date, time]

    except Exception as e:
        print(f"Error in DeepFace/liveness: {e}")

    pygame.display.update()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

cap.release()
pygame.quit()
cv2.destroyAllWindows()


In [ ]:
from deepface import DeepFace

model = DeepFace.find(
    img_path='Sar.jpg',
    db_path='./Database',
    enforce_detection=False,
    model_name='VGG-Face'
)
